In [ ]:
### Import the required libraries
import numpy as np
import matplotlib.pyplot as plt 

import warnings
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
tf.compat.v1.disable_eager_execution()

import os
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

In [ ]:
### Corresponds to grid point index (ny,nx) = (32,58)
costLon  = 240
costLat  = 50

nx = 90
ny = 40
nz = 15

mainDir = '/scratch2/shreyas/global_oce_latlon_4x4'

gridDir = mainDir + '/run_spinup/GRID'

DataDirs = [mainDir + '/run_forward_50yr_dailydump_pk000059400/diags/untarred_output',
             mainDir + '/run_forward_50yr_dailydump_pk000061200/diags/untarred_output']

In [ ]:
#### SAVING DATA INTO XARRAY DATASET ####
#### SAVING XARRAY DATASET AS NC FILE ####

# files_list_TS = []
# files_list_2Datm = []
# init_indices = [594001, 612001]
# length_rec = 18000

# for i in range(length_rec):
#     index = init_indices[0] + i
#     files_list_TS.append(DataDirs[0] + '/' + f'state3d_TS.0000{index:6d}.data')
#     files_list_2Datm.append(DataDirs[0] + '/' + f'state2d_atm.0000{index:6d}.data')
# for i in range(length_rec):
#     index = init_indices[1] + i
#     files_list_TS.append(DataDirs[1] + '/' + f'state3d_TS.0000{index:6d}.data')
#     files_list_2Datm.append(DataDirs[1] + '/' + f'state2d_atm.0000{index:6d}.data')
    
# nt = len(files_list_TS)
# thetaSurf = np.zeros((nt, ny, nx))
# saltSurf  = np.zeros((nt, ny, nx))
# tauX      = np.zeros((nt, ny, nx))
# tauY      = np.zeros((nt, ny, nx))
                     
# for i in range(len(files_list_TS)):
#     thetaSurf[i] = np.reshape(np.fromfile(files_list_TS[i], dtype = '>f')[:nx*ny*nz],
#                               (nz, ny, nx))[0]
#     saltSurf[i]  = np.reshape(np.fromfile(files_list_TS[i], dtype = '>f')[nx*ny*nz:],
#                               (nz, ny, nx))[0]
#     tauX[i]      = np.reshape(np.fromfile(files_list_2Datm[i], dtype = '>f')[7*nx*ny:8*nx*ny],
#                               (ny, nx))
#     tauY[i]      = np.reshape(np.fromfile(files_list_2Datm[i], dtype = '>f')[8*nx*ny:9*nx*ny],
#                               (ny, nx))
    
# # https://docs.xarray.dev/en/stable/generated/xarray.cftime_range.html
# # Allows to filter like this - da_thetaSurf.sel(time=da_thetaSurf.time.dt.month.isin([6, 7, 8]))
# timeDim = xr.cftime_range(start="1651", end = "1751", calendar="360_day")[:-1]
# latDim    = np.arange(-78.,82.,4.)
# lonDim    = np.arange(0.,360,4.)

# da_XC = xr.DataArray(
#     data=np.tile(lonDim, (40,1)),
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Longitude tiled"),
# )

# da_YC = xr.DataArray(
#     data=np.tile(latDim, (90,1)).T,
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Latitude tiled"),
# )

# da_thetaSurf = xr.DataArray(
#     data=thetaSurf,
#     dims=["time", "latitude", "longitude"],
#     coords=dict(
#         time         = timeDim,
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Surface temperature field"),
# )

# da_saltSurf = xr.DataArray(
#     data=saltSurf,
#     dims=["time", "latitude", "longitude"],
#     coords=dict(
#         time         = timeDim,
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Surface salinity field"),
# )

# da_tauX = xr.DataArray(
#     data=tauX,
#     dims=["time", "latitude", "longitude"],
#     coords=dict(
#         time         = timeDim,
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Surface zonal wind stress"),
# )

# da_tauY = xr.DataArray(
#     data=tauY,
#     dims=["time", "latitude", "longitude"],
#     coords=dict(
#         time         = timeDim,
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Surface meridional wind stress"),
# )

# ds = xr.Dataset()
# ds = ds.assign(XC        = da_XC,
#                YC        = da_YC,
#                thetaSurf = da_thetaSurf,
#                saltSurf  = da_saltSurf,
#                tauX      = da_tauX,
#                tauY      = da_tauY)

# hfacc = np.reshape(np.fromfile(gridDir + '/hFacC.data', 
#                               dtype = '>f'), (nz, ny, nx))

# hFacC_mask = hfacc > 0
# hFacC_mask = hFacC_mask.astype(np.float32)
# hFacC_mask = hFacC_mask[0]

# XC = ds['XC'].data
# YC = ds['YC'].data

# latMask = YC > -20.0
# latMask = latMask.astype(float)

# maskFinal = hFacC_mask * latMask
# NaNmaskFinal = np.copy(maskFinal)
# NaNmaskFinal[NaNmaskFinal == 0] = np.nan

# da_hFacC_mask = xr.DataArray(
#     data=hFacC_mask,
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="hFacC mask 2D 1 if > 0, else 0"),
# )

# da_latMask = xr.DataArray(
#     data=latMask,
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Latitude Mask 1 if > -20, else 0"),
# )

# da_maskFinal = xr.DataArray(
#     data=maskFinal,
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Mask 2D 1 if > 0, else 0"),
# )

# da_NaNmaskFinal = xr.DataArray(
#     data=NaNmaskFinal,
#     dims=["latitude", "longitude"],
#     coords=dict(
#         latitude     = latDim,
#         longitude    = lonDim,
#     ),
#     attrs=dict(description="Mask 2D True if > 0, else NaN"),
# )

# ds = ds.assign(hFacC_mask   = da_hFacC_mask,
#           latMask      = da_latMask,
#           maskFinal    = da_maskFinal,
#           NaNmaskFinal = da_NaNmaskFinal)

# wetpoints = np.nonzero(ds['maskFinal'].data)

# ds.to_netcdf('/scratch2/shreyas/LRP_eccov4r4_data/dataGlobalLonLat4x4.nc', mode='w')

In [ ]:
ds = xr.open_dataset('/scratch2/shreyas/LRP_eccov4r4_data/dataGlobalLonLat4x4.nc')

In [ ]:
plt.pcolor(ds['XC'], ds['YC'], 
           ds['NaNmaskFinal']*ds['thetaSurf'].isel(time = 0), 
           cmap = 'jet', vmax = 35)
plt.scatter(240,50,marker = 'x', color = 'brown', label = "Objective function loc")
plt.scatter(232,50,marker = 'o', color = 'black', label = "Nearest wetpoint")
plt.colorbar()

In [ ]:
cost = ds["thetaSurf"].sel(latitude=50.,longitude=240.)
thetaSurfWet = ds["thetaSurf"].data[:, wetpoints[0], wetpoints[1]]
saltSurfWet  = ds["saltSurf"].data[:, wetpoints[0], wetpoints[1]]
tauXWet      = ds["tauX"].data[:, wetpoints[0], wetpoints[1]]
tauYWet      = ds["tauY"].data[:, wetpoints[0], wetpoints[1]]
tauWet       = np.concatenate((tauXWet,tauYWet), axis = 1)

In [ ]:
def quickSetup(X, y, numVars, lrp_methods, lagSteps, baseAnalysis, numAnalysis, **NNkwargs):
    
    result = {}
    
    if lagSteps > 0:
        K = TrainFullyConnectedNN(X[:-lagSteps], y[lagSteps:], **NNkwargs)
        L = TrainLR(X[:-lagSteps], y[lagSteps:], y_ref = 0.0, fit_intercept = False)
    if lagSteps == 0:
        K = TrainFullyConnectedNN(X, y, **NNkwargs)
        L = TrainLR(X, y, y_ref = 0.0, fit_intercept = False)
        
    best_model = K.quickTrain()
    result['cost_predict_NN'] = best_model.predict(X)
    
    regr = L.quickTrain()
    result['cost_predict_LR'] = regr.predict(X)
    
    normalizeDict = {'bool_': True, 'kind': 'MaxAbs'}
    kwargs = {'y_ref': 0.00}

    
    
    for method in lrp_methods:
        
        print(f'Analyze using {method}')
        
        Xplain = XAIR(best_model, method, 'classic', X[baseAnalysis:baseAnalysis+numAnalysis], normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        perVar = int(a.shape[1]/numVars)
        
        rel = np.zeros((numVars, numAnalysis, ny, nx))
        rel[:,:,:,:] = np.nan
        
        for numvar in range(numVars):
            rel[numvar,:,wetpoints[0],wetpoints[1]] = a[:,numvar*perVar:(numvar+1)*perVar].T
        result[method] = rel
        
    print(f'Analyze using lrp.LR')
    
    XL = XLR(regr, X[baseAnalysis:baseAnalysis+numAnalysis])
    a_LR, _ = XL.quick_analyze()
    perVar = int(a_LR.shape[1]/numVars)
    
    rel_LR = np.zeros((numVars, numAnalysis, ny, nx))
    rel_LR[:,:,:,:] = np.nan

    for numvar in range(numVars):
        rel_LR[numvar,:,wetpoints[0],wetpoints[1]] = a_LR[:,numvar*perVar:(numvar+1)*perVar].T
    result['lrp.LR'] = rel_LR
        
    return result

In [ ]:
LayersTheta = [{'size': thetaSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersSalt = [{'size': saltSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersTau = [{'size': tauWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 40                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
Losses = [{'kind': 'mse', 'weight': 1.0}]

LRPDict_theta = {}
LRPDict_salt  = {}
LRPDict_tau   = {}
lagStepsList = [30, 60, 90, 120, 150, 180]

kwargs = {'losses': Losses, 'optim': 'adam', 'metrics': ['mae'],
            'batch_size': 10, 'epochs': 50, 'validation_split': 0.2,
            'filename': 'model', 'dirname': os.path.abspath('')}

for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Salt')
    LRPDict_salt[f'LRP{lagStepsList[i]}'] = quickSetup(saltSurfWet, cost, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0', 'lrp.z'],
                                                       lagSteps = lagStepsList[i], baseAnalysis = 0, numAnalysis = 360,
                                                       layers = LayersSalt, **kwargs)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_a1b0 for salinity (NN trained using salinity only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_salt[f'LRP{lagStepsList[i]}']['lrp.alpha_1_beta_0'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_a1b0 for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_z for salinity (NN trained using salinity only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_salt[f'LRP{lagStepsList[i]}']['lrp.z'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_z for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_LinearRegression for salinity (NN trained using salinity only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_salt[f'LRP{lagStepsList[i]}']['lrp.LR'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_LR for time lag: {lagStepsList[i]} ')

In [ ]:
LayersTheta = [{'size': thetaSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersSalt = [{'size': saltSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersTau = [{'size': tauWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 40                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
Losses = [{'kind': 'mse', 'weight': 1.0}]

LRPDict_theta = {}
LRPDict_salt  = {}
LRPDict_tau   = {}
lagStepsList = [30, 60, 90, 120, 150, 180]

kwargs = {'losses': Losses, 'optim': 'adam', 'metrics': ['mae'],
            'batch_size': 10, 'epochs': 50, 'validation_split': 0.2,
            'filename': 'model', 'dirname': os.path.abspath('')}

for i in range(len(lagStepsList)):
    
    print(f'Lag: {lagStepsList[i]} days, for Theta')
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(thetaSurfWet, cost, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0', 'lrp.z'],
                                                        lagSteps = lagStepsList[i], baseAnalysis = 0, numAnalysis = 360,
                                                        layers = LayersTheta, **kwargs)

In [ ]:
LRPDict_theta[f'LRP30'] = quickSetup(thetaSurfWet, cost, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0', 'lrp.z'],
                                                        lagSteps = 30, baseAnalysis = 0, numAnalysis = 360,
                                                        layers = LayersTheta, **kwargs)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_a1b0 for theta (NN trained using theta only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_theta[f'LRP{lagStepsList[i]}']['lrp.alpha_1_beta_0'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_a1b0 for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_z for theta (NN trained using theta only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_theta[f'LRP{lagStepsList[i]}']['lrp.z'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_z for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_LR for theta (NN trained using theta only)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_theta[f'LRP{lagStepsList[i]}']['lrp.LR'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_LR for time lag: {lagStepsList[i]} ')

In [ ]:
LayersTheta = [{'size': thetaSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersSalt = [{'size': saltSurfWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 20                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
LayersTau = [{'size': tauWet.shape[1], 'activation': None    , 'use_bias': None},
          {'size': 40                   , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1                    , 'activation': 'linear', 'use_bias': False}]
Losses = [{'kind': 'mse', 'weight': 1.0}]

LRPDict_theta = {}
LRPDict_salt  = {}
LRPDict_tau   = {}
lagStepsList = [30, 60, 90, 120, 150, 180]

kwargs = {'losses': Losses, 'optim': 'adam', 'metrics': ['mae'],
            'batch_size': 10, 'epochs': 50, 'validation_split': 0.2,
            'filename': 'model', 'dirname': os.path.abspath('')}

for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Tau')
    LRPDict_tau[f'LRP{lagStepsList[i]}'] = quickSetup(tauWet, cost, numVars = 2, lrp_methods = ['lrp.alpha_1_beta_0', 'lrp.z'],
                                                      lagSteps = lagStepsList[i], baseAnalysis = 0, numAnalysis = 360, 
                                                      layers = LayersTau, **kwargs)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_a1b0 for tauX (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.alpha_1_beta_0'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_a1b0 for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_z for tauX (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.z'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_z for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_LR for tauX (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.LR'][0], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_LR for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_a1b0 for tauY (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.alpha_1_beta_0'][1], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_a1b0 for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_z for tauY (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.z'][1], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_z for time lag: {lagStepsList[i]} ')

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, ax = plt.subplots(2,3)
pltX = 0
pltY = 0
fig.suptitle('LRP_LR for tauY (NN trained using tauX, tauY)', fontsize=16)

for i in range(len(lagStepsList)):
    pltX = int(i/3)
    pltY = int(i%3)
    
    field = np.mean(LRPDict_tau[f'LRP{lagStepsList[i]}']['lrp.LR'][1], axis = 0)
    field = field / np.nanmax(np.abs(field))
    
    p = ax[pltX,pltY].contourf(field, vmin = -1, vmax = 1, levels = 100, cmap = 'RdBu_r')
    ax[pltX,pltY].scatter([58],[32], color = 'black')
    fig.colorbar(p, ax=ax[pltX,pltY])
    ax[pltX,pltY].set_title(f'Mean annual normalized LRP_LR for time lag: {lagStepsList[i]} ')